In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType
import numpy as np
from pyspark.ml.feature import RegexTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from pyspark.sql import functions as sf
import nltk
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA

spark_conf = SparkConf()\
        .setAppName("A2 NLI code")
sc=SparkContext.getOrCreate(spark_conf) 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1589251201859_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
file_schema = StructType([
    StructField("index", StringType(), True),
    StructField("promtID", StringType(), True),
    StructField("pairID", StringType(), True),
    StructField("genre", StringType(), True),
    StructField("sentence1_binary_parse",StringType(),True),
    StructField("sentence2_binary_parse",StringType(),True),
    StructField("sentence1_parse",StringType(),True),
    StructField("sentence2_parse",StringType(),True),
    StructField("sentence1",StringType(),True),
    StructField("sentence2",StringType(),True),
    StructField("label1",StringType(),True),
    StructField("label2",StringType(),True),
    StructField("label3",StringType(),True),
    StructField("label4",StringType(),True),
    StructField("label5",StringType(),True),
    StructField("gold_label",StringType(),True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dev_matched_file = 's3://comp5349-qgao7403/week10/e-2JNSCVIVXV7TULZBDKF4XGBSZ/ta/dev_matched.tsv'
dev_mismatched_flie='s3://comp5349-qgao7403/week10/e-2JNSCVIVXV7TULZBDKF4XGBSZ/ta/dev_mismatched.tsv'
test_matched_file='s3://comp5349-qgao7403/week10/e-2JNSCVIVXV7TULZBDKF4XGBSZ/ta/test_matched.tsv'
test_mismatched_file='s3://comp5349-qgao7403/week10/e-2JNSCVIVXV7TULZBDKF4XGBSZ/ta/test_mismatched.tsv'
train_flie='s3://comp5349-qgao7403/week10/e-2JNSCVIVXV7TULZBDKF4XGBSZ/ta/train.tsv'
my_cols=['genre','sentence1','sentence2']


dev_matched_rdd = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(file_schema).load(dev_matched_file).select(my_cols).cache()
dev_mismatched_rdd = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(file_schema).load(dev_mismatched_flie).select(my_cols).cache()
test_matched_rdd =spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(file_schema).load(test_matched_file).select(my_cols).cache()
test_mismatched_rdd = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(file_schema).load(test_mismatched_file).select(my_cols).cache()
train_rdd = spark.read.format("csv").option("header", "true").option("delimiter", "\t").schema(file_schema).load(train_flie).select(my_cols).cache()




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 3.1.1

In [15]:
merge_matched=dev_matched_rdd.union(test_matched_rdd)
merge_mismatched=dev_mismatched_rdd.union(test_mismatched_rdd)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# import string
# print(string.punctuation)
# table = str.maketrans('.', ' ')
# # table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
# "don't...".translate(table)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
import string
#table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
table = str.maketrans('.', ' ')
def tokenAndMap(row):
    sentence1=row[1]
    sentence2=row[2]

    if sentence2 is None:
        sentence = sentence1
    elif sentence1 is None:
        sentence = sentence2
    else:
        sentence = sentence1+' '+sentence2
    # get rid of the punctuation '.' and lower sentence
    words = nltk.word_tokenize(sentence.translate(table).lower()) 
    return [(x,1) for x in words]

#extract the words by nltk tools and map them and distinct them
dist_m_matched = merge_matched.rdd.flatMap(tokenAndMap).distinct()
dist_m_mismatched = merge_mismatched.rdd.flatMap(tokenAndMap).distinct()


# join them on common words then count 
num_comm_words=dist_m_matched.join(dist_m_mismatched).count()

# unique number = tatal number - common number 
num_uniq_words_matched=dist_m_matched.count()-num_comm_words
num_uniq_words_mismatched=dist_m_mismatched.count()-num_comm_words




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
print('the number of common words betweenmatchedandmismatchedsets:',
      num_comm_words,
      '\nthe number of words unique to the matched sets; ',
      num_uniq_words_matched,
      '\nthe number of words unique to the mismatched sets: ',
      num_uniq_words_mismatched)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

the number of common words betweenmatchedandmismatchedsets: 9448 
the number of words unique to the matched sets;  10462 
the number of words unique to the mismatched sets:  8064

# 3.1.2
## without stopword dict

In [19]:
train_rdd.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+
|     genre|           sentence1|           sentence2|
+----------+--------------------+--------------------+
|government|Conceptually crea...|Product and geogr...|
| telephone|you know during t...|You lose the thin...|
|   fiction|One of our number...|A member of my te...|
+----------+--------------------+--------------------+
only showing top 3 rows

In [20]:
def token_Map_word_genre(row):
    genre=row[0]
    sentence1=row[1]
    sentence2=row[2]

    if sentence2 is None:
        sentence = sentence1
    elif sentence1 is None:
        sentence = sentence2
    else:
        sentence = sentence1+' '+sentence2
    # get rid of the punctuation '.' and lower sentence
    words = nltk.word_tokenize(sentence.translate(table).lower()) 
    return [(x,genre) for x in words]

dict_word_genre=train_rdd.rdd.flatMap(token_Map_word_genre).distinct().countByKey()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
total=len(dict_word_genre)
rdd_train=sc.parallelize(list(dict_word_genre.items()))

total_5=rdd_train.filter(lambda a: True if a[1]==5 else False).count()/total*100
total_4=rdd_train.filter(lambda a: True if a[1]==4 else False).count()/total*100
total_3=rdd_train.filter(lambda a: True if a[1]==3 else False).count()/total*100
total_2=rdd_train.filter(lambda a: True if a[1]==2 else False).count()/total*100
total_1=rdd_train.filter(lambda a: True if a[1]==1 else False).count()/total*100


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:

print(  'percentages of words appearing in 5 genres: {:.2f}%'.format(total_5),
        '\npercentages of words appearing in 4 genres: {:.2f}%'.format(total_4),
      '\npercentages of words appearing in 3 genres: {:.2f}%'.format(total_3),
      '\npercentages of words appearing in 2 genres: {:.2f}%'.format(total_2),
      '\npercentages of words appearing in 1 genres: {:.2f}%'.format(total_1),)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

percentages of words appearing in 5 genres: 9.15% 
percentages of words appearing in 4 genres: 6.28% 
percentages of words appearing in 3 genres: 8.02% 
percentages of words appearing in 2 genres: 13.61% 
percentages of words appearing in 1 genres: 62.95%

## with stopword dict

In [23]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def token_stopword_Map_word_genre(row):
    genre=row[0]
    sentence1=row[1]
    sentence2=row[2]

    if sentence2 is None:
        sentence = sentence1
    elif sentence1 is None:
        sentence = sentence2
    else:
        sentence = sentence1+' '+sentence2
    # get rid of the punctuation '.' and lower sentence
    words = nltk.word_tokenize(sentence.translate(table).lower()) 
    # get rid of stopword
    words = [w for w in words if not w in stop_words]
    return [(x,genre) for x in words]

dict_word_genre=train_rdd.rdd.flatMap(token_stopword_Map_word_genre).distinct().countByKey()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
total=len(dict_word_genre)
rdd_train=sc.parallelize(list(dict_word_genre.items()))

total_5=rdd_train.filter(lambda a: True if a[1]==5 else False).count()/total*100
total_4=rdd_train.filter(lambda a: True if a[1]==4 else False).count()/total*100
total_3=rdd_train.filter(lambda a: True if a[1]==3 else False).count()/total*100
total_2=rdd_train.filter(lambda a: True if a[1]==2 else False).count()/total*100
total_1=rdd_train.filter(lambda a: True if a[1]==1 else False).count()/total*100

print(  'percentages of words appearing in 5 genres: {:.2f}%'.format(total_5),
        '\npercentages of words appearing in 4 genres: {:.2f}%'.format(total_4),
      '\npercentages of words appearing in 3 genres: {:.2f}%'.format(total_3),
      '\npercentages of words appearing in 2 genres: {:.2f}%'.format(total_2),
      '\npercentages of words appearing in 1 genres: {:.2f}%'.format(total_1),)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

percentages of words appearing in 5 genres: 9.01% 
percentages of words appearing in 4 genres: 6.28% 
percentages of words appearing in 3 genres: 8.03% 
percentages of words appearing in 2 genres: 13.63% 
percentages of words appearing in 1 genres: 63.05%

## 3.2

## Flat Data

In [9]:
def extractAllSentences(row):
    genre=row[0]
    sentence1=row[1]
    sentence2=row[2]
    if sentence2 is None:
        sentence = sentence1
    elif sentence1 is None:
        sentence = sentence2
    else:
        sentence = sentence1+' '+sentence2
    return (genre,sentence)
    
flat_train_rdd=train_rdd.rdd.map(extractAllSentences).repartition(5).cache()
# fit_data_limit=train_rdd.limit(20000).rdd.map(extractAllSentences)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
flat_train_rdd.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

789904

## Using Google encode

In [11]:
def sentenece_embed(sen_text_partition):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)
    # mapPartition would supply element inside a partition using generator stype
    # this does not fit tensorflow stype
    sen_text_list = [text[1] for text in sen_text_partition]
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(sen_text_list))
   
    return message_embeddings

sentence_embedding = flat_train_rdd.mapPartitions(sentenece_embed).cache()
# fit_sentence_embedding=fit_data_limit.mapPartitions(sentenece_embed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sentence_embedding.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5

## Transfer embeded data to Vector

In [13]:


sentence_embedding_df = spark.createDataFrame(sentence_embedding.map(lambda v: v.tolist()))
# fit_sentence_embedding_df = spark.createDataFrame(fit_sentence_embedding.map(lambda v: v.tolist()))

assembler = VectorAssembler(inputCols=sentence_embedding_df.columns,
    outputCol="features")

# fit_assembler=VectorAssembler(inputCols=fit_sentence_embedding_df.columns,
#     outputCol="features")

sentence_embedding_vectors = assembler.transform(sentence_embedding_df).select("features")
# fit_sentence_embedding_vectors = fit_assembler.transform(fit_sentence_embedding_df).select("features")

# sentence_embedding_vectors.take(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# sentence_embedding_vectors.rdd.getNumPartitions()
# fit_data=sentence_embedding_vectors.repartition(200)

## Using PCA to reduce dimensions

In [14]:
pca = PCA(k=100, inputCol="features", outputCol="pca")
model = pca.fit(sentence_embedding_vectors)
pca_result = model.transform(sentence_embedding_vectors).select('pca')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# pca_result = model.transform(sentence_embedding_vectors).select('pca')
# fit_pca_result = model.transform(fit_sentence_embedding_vectors).select('pca')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Using Kmeans to clustering the sentences

In [15]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(featuresCol='pca',k=5)
model=kmeans.fit(pca_result)
predictions = model.transform(pca_result)
centers = model.clusterCenters()
predictions.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+
|                 pca|prediction|
+--------------------+----------+
|[-0.4034157047051...|         3|
|[-0.2721225915225...|         3|
|[0.30520967715876...|         0|
|[0.27304168878753...|         1|
|[-0.4300581487699...|         3|
+--------------------+----------+
only showing top 5 rows

In [24]:
# from pyspark.ml.clustering import GaussianMixture
# gmm=GaussianMixture(featuresCol='pca', predictionCol='prediction', k=5, probabilityCol='probability', tol=0.01, maxIter=100, seed=None)
# cmodel=gmm.fit(pca_result)
# predictions = model.transform(pca_result)
# predictions.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Show confusion_matrix and classification_report

In [16]:
local_kmeans_data= predictions.select('prediction').rdd.map(lambda row:(row[0]))

#[genre, senetence, word2vec, pca, prediction, majority_abel]

local_kmeans_data_index=local_kmeans_data.zipWithIndex().map(lambda row:(row[1],row[0]))

train_genre_index=flat_train_rdd.map(lambda row : (row[0])).zipWithIndex().map(lambda row:(row[1],row[0]))

zip_genre_kmeanpred=train_genre_index.join(local_kmeans_data_index).values()

### zip_genre_kmeanpred: [ genre, prediction]

zip_genre_kmeanpred.count()

label_0=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 0==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_1=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 1==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_2=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 2==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_3=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 3==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_4=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 4==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]



# print(label_0,
# label_1,
# label_2,
# label_3,
# label_4)

def addLableMap(row):
    labelId=row[1]
    if 0==labelId:
        label=label_0
    elif 1==labelId:
        label=label_1
    elif 2==labelId:
        label=label_2    
    elif 3==labelId:
        label=label_3    
    elif 4==labelId:
        label=label_4
    
    return (row[0],labelId,label)
    
    
zip_genre_kmeanpred_predlabel=zip_genre_kmeanpred.map(addLableMap)

Y_true=zip_genre_kmeanpred_predlabel.map(lambda row:(row[0])).collect()
Y_pred=zip_genre_kmeanpred_predlabel.map(lambda row:(row[2])).collect()



print(classification_report(Y_true, Y_pred))
print('True Label: ',"fiction", "government", "slate","telephone","travel")
print(confusion_matrix( Y_pred,Y_true,labels=["fiction", "government", "slate","telephone","travel"],normalize='pred'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

     fiction       0.31      0.56      0.40    159196
  government       0.37      0.51      0.43    154700
       slate       0.00      0.00      0.00    154612
   telephone       0.36      0.33      0.34    166696
      travel       0.47      0.42      0.44    154700

    accuracy                           0.36    789904
   macro avg       0.30      0.36      0.32    789904
weighted avg       0.30      0.36      0.32    789904

True Label:  fiction government slate telephone travel
[[0.55898389 0.24996122 0.39022198 0.35783102 0.25122818]
 [0.16504184 0.50513898 0.29387758 0.19494769 0.17098255]
 [0.         0.         0.         0.         0.        ]
 [0.15688208 0.13371687 0.1890927  0.33005591 0.15471881]
 [0.11909219 0.11118293 0.12680775 0.11716538 0.42307046]]
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

## Using TFIDF to encode sentences

In [12]:
from pyspark.mllib.feature import HashingTF, IDF
# Load documents (one per line).
# documents = sc.textFile("data/mllib/kmeans_data.txt").map(lambda line: line.split(" "))
documents=flat_train_rdd.map(lambda row:row[1])
hashingTF = HashingTF(10000)
tf = hashingTF.transform(documents)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Using mllib Kmeans to clustering the sentences

In [13]:
from pyspark.mllib.clustering import KMeans, KMeansModel

clusters = KMeans.train(tfidfIgnore, 5, maxIterations=1000, initializationMode="k-means||")
preditions=clusters.predict(tfidfIgnore)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Show confusion_matrix and classification_report

In [14]:
#local_kmeans_data= predictions.select('prediction').rdd.map(lambda row:(row[0]))
local_kmeans_data=preditions
#[genre, senetence, word2vec, pca, prediction, majority_abel]

local_kmeans_data_index=local_kmeans_data.zipWithIndex().map(lambda row:(row[1],row[0]))

train_genre_index=flat_train_rdd.map(lambda row : (row[0])).zipWithIndex().map(lambda row:(row[1],row[0]))

zip_genre_kmeanpred=train_genre_index.join(local_kmeans_data_index).values()

### zip_genre_kmeanpred: [ genre, prediction]

zip_genre_kmeanpred.count()

label_0=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 0==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_1=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 1==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_2=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 2==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_3=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 3==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]
label_4=sorted(list(zip_genre_kmeanpred.filter(lambda row: True if 4==row[1]  else False).countByKey().items()),key=lambda x:x[1],reverse=True)[0][0]



def addLableMap(row):
    labelId=row[1]
    if 0==labelId:
        label=label_0
    elif 1==labelId:
        label=label_1
    elif 2==labelId:
        label=label_2    
    elif 3==labelId:
        label=label_3    
    elif 4==labelId:
        label=label_4
    
    return (row[0],labelId,label)
    
    
zip_genre_kmeanpred_predlabel=zip_genre_kmeanpred.map(addLableMap)

Y_true=zip_genre_kmeanpred_predlabel.map(lambda row:(row[0])).collect()
Y_pred=zip_genre_kmeanpred_predlabel.map(lambda row:(row[2])).collect()



print(classification_report(Y_true, Y_pred))


# print(label_0,
# label_1,
# label_2,
# label_3,
# label_4)

print('True Label: ',"fiction", "government", "slate","telephone","travel")
print(confusion_matrix(Y_pred,Y_true,labels=["fiction", "government", "slate","telephone","travel"],normalize='pred'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

              precision    recall  f1-score   support

     fiction       0.00      0.00      0.00     77348
  government       0.26      0.03      0.05     77350
       slate       0.24      0.07      0.11     77306
   telephone       0.23      0.79      0.35     83348
      travel       0.23      0.21      0.22     77350

    accuracy                           0.23    392702
   macro avg       0.19      0.22      0.15    392702
weighted avg       0.19      0.23      0.15    392702

True Label:  fiction government slate telephone travel
[[0.         0.         0.         0.         0.        ]
 [0.01683301 0.02692954 0.02043826 0.01700101 0.02253394]
 [0.05264519 0.06095669 0.07466432 0.0522748  0.06599871]
 [0.78334281 0.70682612 0.72515717 0.78595767 0.70133161]
 [0.14717898 0.20528765 0.17974025 0.14476652 0.21013575]]
/usr/local/lib64/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 